<a href="https://colab.research.google.com/github/TeneikaAskew/BWD2022/blob/main/Regex_Check.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [14]:
import re
from typing import List, Dict


def format_response(raw: str) -> Dict[str, any]:
    if not raw:
        return {"elevatorPitch": "", "themes": [], "explanation": ""}

    extracted_content = {
        "elevatorPitch": "",
        "themes": [],
        "explanation": ""
    }

    # Normalize text
    # text = raw.replace("\r\n", "\n").strip()
    text = raw.replace("\\n", "\n").replace("\r\n", "\n").strip()
    text = re.sub(r"Here are the three key outputs:?", "", text, flags=re.I)
    text = re.sub(r"Here are three key outputs:?", "", text, flags=re.I)
    text = re.sub(r"based on the provided resume text and basic analysis:", "", text, flags=re.I)

    # Elevator pitch patterns
    elevator_patterns = [
      r"(?:\*\*)?Professional Elevator Pitch(?:\*\*)?:?\s*[\r\n]+([\s\S]*?)(?=\n\s*(?:\*\*)?(?:Three Specific |Key )?Improvement Themes|(?:\*\*)?Improvement Themes|\n\s*\d+\.|\n\s*\*\*|$)",
      r"(?:\*\*)?Elevator Pitch(?:\*\*)?:?\s*[\r\n]+([\s\S]*?)(?=\n\s*(?:\*\*)?(?:Three Specific |Key )?Improvement Themes|(?:\*\*)?Improvement Themes|\n\s*\d+\.|\n\s*\*\*|$)",
      r"A professional elevator pitch based on the resume text:?\s*[\r\n]+([\s\S]*?)(?=\n\s*(?:Three specific improvement themes|Improvement Themes|2\.|Resume Grade|$))",
      r"1\.\s*Professional (?:E|e)levator (?:P|p)itch:?\s*[\r\n]+([\s\S]*?)(?=\n\s*\d+\.|\n\s*$)",
      r"\*\*Professional Elevator Pitch:\*\*\s*[\r\n]+([\s\S]*?)(?=\n\s*\*\*|\n\s*\d+\.|\n\s*$)",
      r"1\.\s*Professional Elevator Pitch:?\s*[\r\n]+([\s\S]*?)(?=\n\s*\d+\.|\n\s*$)",
      r"Professional Elevator Pitch:?\s*[\r\n]+([\s\S]*?)(?=\n\s*\*\*|\n\s*\d+\.|\n\s*$)",
      r"^Elevator Pitch:?\s*[\r\n]+([\s\S]*?)(?=\n\s*\*\*|\n\s*\d+\.|\n\s*$)",
      r"Here are the three key outputs:\s*\\n+([\s\S]{40,500}?)(?=\\n+Here are|\\n+(?:Three|three) specific improvement themes|\\n+The resume|\\n+\*\*|\\n+$)",
      r"^As\s+a\s+[^\n]{10,250}?(?=\n{2,}|\nHere are|\nThree|\nThe resume|\nResume Grade|$)",
      r"Here are the three key outputs:\s*\n+([^\n]{40,500}.*?)(?=\n{2,}|Here are|Three specific improvement themes|The resume|Resume Grade|$)",
      r"([A-Z][\s\S]{30,800}?)(?=\n{2,}|\n+(Here are|Three|The resume|Resume Grade|A brief explanation|Explanation|Improvement Themes|$))"
    ]


    for pattern in elevator_patterns:
        match = re.search(pattern, text, flags=re.I | re.M)
        if match:
            pitch = match.group(1).strip()
            pitch = re.sub(r"^\*\*\s*|\s*\*\*$", "", pitch)
            pitch = re.sub(r"\n+", " ", pitch)
            pitch = re.sub(r"\s+", " ", pitch).strip()
            extracted_content["elevatorPitch"] = pitch
            break

    # Improvement themes patterns
    theme_patterns = [
      r"\*\*Three Specific Improvement Themes:\*\*\s*([\s\S]*?)(?=\n\s*\*\*Resume Grade|\n\s*$)",
      r"(?:\*\*)?(?:Three Specific )?Improvement Themes(?:\*\*)?:?\s*([\s\S]*?)(?=\n\s*(?:\*\*)?(?:Resume Grade Explanation|Resume Grade)|\n\s*(?:\*\*)?Resume Grade:|\n\s*$)",

      r"2\.\s*(?:Three Specific )?Improvement Themes:?\s*([\s\S]*?)(?=\n\s*\d+\.|\n\s*$)",
      r"(?:\*\*)?Key Improvement Themes(?:\*\*)?:?\s*([\s\S]*?)(?=\n\s*(?:\*\*)?(?:Resume Grade Explanation|Resume Grade)|\n\s*$)",
      r"Three specific improvement themes:?\s*([\s\S]*?)(?=\n\s*(?:A brief explanation of the resume grade|Resume Grade|3\.|$))",
      r"Three Specific Improvement Themes:?\s*([\s\S]*?)(?=\n\s*\*\*|\n\s*\d+\.|\n\s*$)",
      r"\*\*Improvement Themes:\*\*\s*([\s\S]*?)(?=\n\s*\*\*|\n\s*\d+\.|\n\s*$)",
      r"Improvement Themes:?\s*([\s\S]*?)(?=\n\s*\*\*|\n\s*\d+\.|\n\s*$)",
      r"2\.\s*[Tt]hree specific improvement themes:?\s*([\s\S]*?)(?=\n\s*3\.|\n\s*\*\*Brief Explanation|\n\s*Resume Grade|$)",
      r":[ \t]*([\s\S]*?)(?=\n+(?:The resume|Resume Grade|A brief explanation|Explanation|$))",
      r"^(?=.*\b(improve|strengthen|enhance|suggestions)\b)[^\n:]{5,250}:\s*([\s\S]*?)(?=\n+(?:The resume|Resume Grade|A brief explanation|Explanation|$))"

    ]


    for pattern in theme_patterns:
        match = re.search(pattern, text, flags=re.I | re.M)
        if match:
            themes_text = match.group(1).strip()
            themes = []

            # Try extracting structured bullet points or lines
            lines = themes_text.split("\n")
            current_theme = ""
            for line in lines:
                line = line.strip()
                if not line:
                    continue
                if re.match(r"^[\d]+\.", line) or re.match(r"^[\*\-•]", line):
                    if current_theme:
                        themes.append(current_theme.strip())
                    current_theme = re.sub(r"^[\d]+\.\s*|^[\*\-•]\s*", "", line)
                else:
                    current_theme += " " + line
            if current_theme:
                themes.append(current_theme.strip())

            # Clean up extracted themes
            themes = [re.sub(r"\*\*", "", theme).strip() for theme in themes if len(theme.strip()) > 5]

            # 🔁 Fallback: If no themes were split (single long blob), try line-by-line
            if len(themes) <= 1 and '\n' in themes_text:
                themes = []
                for line in themes_text.split('\n'):
                    cleaned = line.strip().replace("•", "").replace("*", "").replace("-", "")
                    if len(cleaned) > 10:
                        themes.append(cleaned)

            themes = [re.sub(r"\*\*", "", theme).strip() for theme in themes if len(theme.strip()) > 5]
            extracted_content["themes"] = themes
            break


    explanation_patterns = [
        r"(The resume grade of [A-F][+-]?\s+[^.]+\.[\s\S]*?(?=\n\s*$|$))" ,
        r"\*\*Resume Grade:\*\*\s*([\s\S]*?)(?=\n\s*\*\*|\n\s*$|$)",
        r"(?:\*\*)?Resume Grade(?:\*\*)?:?\s*\n?([\s\S]*?)(?=\n\s*$|$)",
        r"(?:\*\*)?Brief (?:E|e)xplanation of the (?:R|r)esume (?:G|g)rade(?:\*\*)?:?\s*\n?([\s\S]*?)(?=\n\s*$|$)",
        r"3\.\s*(?:Brief )?(?:E|e)xplanation of the (?:R|r)esume (?:G|g)rade:?\s*\n?([\s\S]*?)(?=\n\s*$|$)",
        r"\*\*Resume Grade Explanation:\*\*\s*([\s\S]*?)(?=\n\s*\*\*|\n\s*\d+\.|\n\s*$)",
        r"Resume Grade Explanation:?\s*([\s\S]*?)(?=\n\s*\*\*|\n\s*\d+\.|\n\s*$)",
        r"3\.\s*Explanation:?\s*([\s\S]*?)(?=\n\s*\d+\.|\n\s*$)",
        r"(The resume grade of [A-F][+-]?\s+[^.]+\.[\s\S]*?(?=\n\s*$|$))"
    ]


    for pattern in explanation_patterns:
        match = re.search(pattern, text, flags=re.I | re.M)
        if match:
            explanation = match.group(1).strip()
            explanation = re.sub(r"^\*\*\s*|\s*\*\*$", "", explanation)
            explanation = re.sub(r"\n+", " ", explanation)
            explanation = re.sub(r"\s+", " ", explanation).strip()
            extracted_content["explanation"] = explanation
            break

    return extracted_content


#Test 1

In [15]:
raw = '''Here are the three key outputs:\n\nAs a seasoned IT management professional with over 5 years of experience, I excel in driving results in IT support, incident management, project management, and network administration, with a strong focus on cybersecurity and IAM. With expertise in security certification testing, vulnerability management, and security assessments, I deliver high-quality solutions and foster strong working relationships with teams, management, and clients.\
n\nTo improve, the candidate should focus on: \n Quantifying achievements with specific numbers and metrics to demonstrate impact, \n Providing more context on job responsibilities to clarify scope and relevance, \n Developing a clear career progression and summary to enhance narrative flow.
\n\nThe resume grade of B reflects a solid foundation in IT management and cybersecurity, with strong technical skills and relevant experience, but also reveals areas for improvement in quantifying achievements and clarifying job responsibilities. Overall, the resume effectively showcases the candidate's strengths, but could benefit from refinement to elevate it to a more distinguished level.\n",
'''

result = format_response(raw)
print("Elevator Pitch:\n", result['elevatorPitch'])
print("\nImprovement Themes:")
for idx, theme in enumerate(result['themes'], 1):
    print(f"{idx}. {theme}")
print("\nResume Grade Explanation:\n", result['explanation'])

Elevator Pitch:
 As a seasoned IT management professional with over 5 years of experience, I excel in driving results in IT support, incident management, project management, and network administration, with a strong focus on cybersecurity and IAM. With expertise in security certification testing, vulnerability management, and security assessments, I deliver high-quality solutions and foster strong working relationships with teams, management, and clients.n To improve, the candidate should focus on: Quantifying achievements with specific numbers and metrics to demonstrate impact, Providing more context on job responsibilities to clarify scope and relevance, Developing a clear career progression and summary to enhance narrative flow.

Improvement Themes:
1. Quantifying achievements with specific numbers and metrics to demonstrate impact,
2. Providing more context on job responsibilities to clarify scope and relevance,
3. Developing a clear career progression and summary to enhance narrat

In [ ]:
raw = '''Here are the three key outputs based on the provided resume text and basic analysis:
\n\n**Professional Elevator Pitch:** \nResults-driven Security Program Manager with 5+ years of experience in optimizing program management, sustainment, and compliance within multinational liaison offices. Proven track record in streamlining complex operations and driving continuous improvement.
\n\n**Three Specific Improvement Themes:**\n1. To further enhance bullet points, consider adding more specific metrics or data to support claims, such as \"Improved cross-functional coordination by 25% through clear, consistent correspondence across multiple agencies.\"\n2. There are some instances of overly wordy phrases or sentences; revising these sections can help improve clarity and concision.\n3. Consider reorganizing bullet points to group similar skills or experiences together for easier scanning.
\n\n**Resume Grade:** \nThe resume earns a grade of A (100%), reflecting its clear, concise writing style and robust bullet points that effectively demonstrate the candidate's experience and skills in security program management.\n",

'''

result = format_response(raw)
print("Elevator Pitch:\n", result['elevatorPitch'])
print("\nImprovement Themes:")
for idx, theme in enumerate(result['themes'], 1):
    print(f"{idx}. {theme}")
print("\nResume Grade Explanation:\n", result['explanation'])


Elevator Pitch:
 Results-driven Security Program Manager with 5+ years of experience in optimizing program management, sustainment, and compliance within multinational liaison offices. Proven track record in streamlining complex operations and driving continuous improvement.

Improvement Themes:
1. To further enhance bullet points, consider adding more specific metrics or data to support claims, such as "Improved cross-functional coordination by 25% through clear, consistent correspondence across multiple agencies."
2. There are some instances of overly wordy phrases or sentences; revising these sections can help improve clarity and concision.
3. Consider reorganizing bullet points to group similar skills or experiences together for easier scanning.

Resume Grade Explanation:
 The resume earns a grade of A (100%), reflecting its clear, concise writing style and robust bullet points that effectively demonstrate the candidate's experience and skills in security program management.


In [ ]:
raw = '''Here are the three key outputs:
\n\n1. Professional elevator pitch: \nAs a seasoned Security Program Manager with over 5 years of experience in multinational liaison offices, I excel at streamlining complex operations and driving continuous improvement. With expertise in logistics, data-driven marketing, and compliance, I deliver strategic solutions that capitalize on emerging opportunities.
\n\n2. Three specific improvement themes:\n- Consider adding more quantitative metrics to bullet points to enhance impact, as some bullets lack concrete numbers or percentages.\n- There is room for expansion on soft skills and accomplishments beyond technical expertise, such as team leadership, stakeholder engagement, or mentoring experiences.\n- Review formatting consistency throughout the resume, particularly in bullet points, to ensure easy readability.
\n\n3. Brief explanation of the resume grade: \nThis resume scores an A (92.9%) due to its clear structure, concise language, and strong accomplishments showcasing expertise in security program management. However, minor improvements in formatting and emphasis on soft skills could further enhance the overall impact of the resume.\n"
'''

result = format_response(raw)
print("Elevator Pitch:\n", result['elevatorPitch'])
print("\nImprovement Themes:")
for idx, theme in enumerate(result['themes'], 1):
    print(f"{idx}. {theme}")
print("\nResume Grade Explanation:\n", result['explanation'])


Elevator Pitch:
 As a seasoned Security Program Manager with over 5 years of experience in multinational liaison offices, I excel at streamlining complex operations and driving continuous improvement. With expertise in logistics, data-driven marketing, and compliance, I deliver strategic solutions that capitalize on emerging opportunities.

Improvement Themes:
1. Consider adding more quantitative metrics to bullet points to enhance impact, as some bullets lack concrete numbers or percentages.
2. There is room for expansion on soft skills and accomplishments beyond technical expertise, such as team leadership, stakeholder engagement, or mentoring experiences.
3. Review formatting consistency throughout the resume, particularly in bullet points, to ensure easy readability.

Resume Grade Explanation:
 This resume scores an A (92.9%) due to its clear structure, concise language, and strong accomplishments showcasing expertise in security program management. However, minor improvements in f

In [ ]:
raw = '''Here are the three key outputs:
\n\n**Professional Elevator Pitch:**\nAs an accomplished Security Program Manager with over 5 years of experience in optimizing program management, logistics, and compliance, I excel at streamlining complex operations and setting measurable goals. My expertise in driving continuous improvement and delivering strategic solutions makes me a valuable asset to any organization.
\n\n**Three Specific Improvement Themes:**\n\n1. **Enhance Quantifiable Metrics:** Some bullet points lack specific numbers or metrics, making it harder to evaluate the impact of accomplishments; adding more quantifiable data would strengthen these claims.\n2. **Varied Action Verbs:** Repeating action verbs like \"Improved\" and \"Reduced\" makes the text less engaging; incorporating a mix of action verbs would enhance readability and create a dynamic tone.\n3. **Bullet Point Organization:** Some bullet points seem unrelated to each other, creating disjointedness in the narrative; reorganizing them into logical groups or categories would improve overall flow.
\n\n**Resume Grade:**\nBased on the analysis, I would give this resume an A (92.9%) due to its strong accomplishments, clear formatting, and professional tone. The writer has demonstrated excellent expertise in security program management and has presented a well-structured narrative that effectively communicates their skills and achievements.\n"
'''

result = format_response(raw)
print("Elevator Pitch:\n", result['elevatorPitch'])
print("\nImprovement Themes:")
for idx, theme in enumerate(result['themes'], 1):
    print(f"{idx}. {theme}")
print("\nResume Grade Explanation:\n", result['explanation'])


Elevator Pitch:
 As an accomplished Security Program Manager with over 5 years of experience in optimizing program management, logistics, and compliance, I excel at streamlining complex operations and setting measurable goals. My expertise in driving continuous improvement and delivering strategic solutions makes me a valuable asset to any organization.

Improvement Themes:
1. Enhance Quantifiable Metrics: Some bullet points lack specific numbers or metrics, making it harder to evaluate the impact of accomplishments; adding more quantifiable data would strengthen these claims.
2. Varied Action Verbs: Repeating action verbs like "Improved" and "Reduced" makes the text less engaging; incorporating a mix of action verbs would enhance readability and create a dynamic tone.
3. Bullet Point Organization: Some bullet points seem unrelated to each other, creating disjointedness in the narrative; reorganizing them into logical groups or categories would improve overall flow.

Resume Grade Expla

In [ ]:
raw = '''Here are the three key outputs:
\n\n**Professional Elevator Pitch:**\nAs an accomplished Security Program Manager with over 5 years of experience in optimizing program management, logistics, and compliance, I excel at streamlining complex operations and setting measurable goals. My expertise in driving continuous improvement and delivering strategic solutions makes me a valuable asset to any organization.
\n\n**Three Specific Improvement Themes:**\n\n1. **Enhance Quantifiable Metrics:** Some bullet points lack specific numbers or metrics, making it harder to evaluate the impact of accomplishments; adding more quantifiable data would strengthen these claims.\n2. **Varied Action Verbs:** Repeating action verbs like \"Improved\" and \"Reduced\" makes the text less engaging; incorporating a mix of action verbs would enhance readability and create a dynamic tone.\n3. **Bullet Point Organization:** Some bullet points seem unrelated to each other, creating disjointedness in the narrative; reorganizing them into logical groups or categories would improve overall flow.
\n\n**Resume Grade:**\nBased on the analysis, I would give this resume an A (92.9%) due to its strong accomplishments, clear formatting, and professional tone. The writer has demonstrated excellent expertise in security program management and has presented a well-structured narrative that effectively communicates their skills and achievements.\n"
'''

result = format_response(raw)
print("Elevator Pitch:\n", result['elevatorPitch'])
print("\nImprovement Themes:")
for idx, theme in enumerate(result['themes'], 1):
    print(f"{idx}. {theme}")
print("\nResume Grade Explanation:\n", result['explanation'])


Elevator Pitch:
 As an accomplished Security Program Manager with over 5 years of experience in optimizing program management, logistics, and compliance, I excel at streamlining complex operations and setting measurable goals. My expertise in driving continuous improvement and delivering strategic solutions makes me a valuable asset to any organization.

Improvement Themes:
1. Enhance Quantifiable Metrics: Some bullet points lack specific numbers or metrics, making it harder to evaluate the impact of accomplishments; adding more quantifiable data would strengthen these claims.
2. Varied Action Verbs: Repeating action verbs like "Improved" and "Reduced" makes the text less engaging; incorporating a mix of action verbs would enhance readability and create a dynamic tone.
3. Bullet Point Organization: Some bullet points seem unrelated to each other, creating disjointedness in the narrative; reorganizing them into logical groups or categories would improve overall flow.

Resume Grade Expla

In [ ]:
import pandas as pd

# Load the uploaded CSV file
file_path = '/content/supabase-logs-siuqvhscuiycvdrtiqsh.csv (2).csv'
df = pd.read_csv(file_path)

# Display the first few rows to confirm structure and column names
df.head()


,event_message,event_type,function_id,id,level,timestamp
0,AI Response: Here are the three key outputs:\...,Log,f880a056-93cf-4ba0-af8b-3155334da591,b5dd55c2-d07a-4668-9549-7268fe8cb4c8,log,1745423756697610
1,AI Response: Here are the three key outputs:\...,Log,f880a056-93cf-4ba0-af8b-3155334da591,88c4a93a-7ebe-44a1-84e7-9df22911df62,log,1745423752179668
2,AI Response: Here are the three key outputs:\...,Log,f880a056-93cf-4ba0-af8b-3155334da591,f19b8326-f5de-4813-9ac6-9cc926c08c81,log,1745423708707722
3,AI Response: Here are the three key outputs:\...,Log,f880a056-93cf-4ba0-af8b-3155334da591,bef7ce90-10ce-4202-ae5b-a87dbfe768e5,log,1745423704896735
4,AI Response: Here are the three key outputs b...,Log,f880a056-93cf-4ba0-af8b-3155334da591,17163a85-5a8b-4ae5-80cb-609e83235cca,log,1745420960637821


In [ ]:
results = df['event_message'].apply(format_response)
df = df['event_message']
results_df = pd.DataFrame(results.tolist())
results.reset_index(drop=True, inplace=True) # Reset index
results_df = pd.concat([df, results_df], axis=1) # Concatenate DataFrames

In [ ]:
results_df

,event_message,elevatorPitch,themes,explanation
0,AI Response: Here are the three key outputs:\...,,[The resume could benefit from more specific e...,The resume grade of B reflects a good foundati...
1,AI Response: Here are the three key outputs:\...,,[are: The resume requires a more professional ...,of D+ indicates that the resume requires signi...
2,AI Response: Here are the three key outputs:\...,,[are: The resume requires a more professional ...,of D indicates that the candidate's resume req...
3,AI Response: Here are the three key outputs:\...,,[The resume could benefit from more descriptiv...,of B indicates that while Sally has a good fou...
4,AI Response: Here are the three key outputs b...,,"[Quantify achievements: Instead of stating ""Sh...",With a grade of D (56.2%) and several areas fo...
5,AI Response: Here are the three key outputs:\...,As a creative and outgoing individual with exp...,[Refine objective statement to focus on specif...,This resume earns a D grade due to its lack of...
6,AI Response: Here are the three key outputs:\...,,[Quantify accomplishments by incorporating spe...,Explanation:
7,AI Response: Here are the three key outputs b...,"Hi, I'm Sally Sad, a creative and social indiv...",[Professionalism and Tone: The resume's inform...,This resume scores a D (49.8%) due to its unpr...
8,AI Response: Here are the three key outputs:\...,Sally is a creative and social individual with...,[Quantify accomplishments: Instead of just lis...,Explanation
9,AI Response: Here are the three key outputs:\...,As a creative and social individual with exper...,"[Quantify achievements: Instead of stating ""Sh...",Explanation:


#Result 1

In [ ]:
from google.colab import sheets
sheet = sheets.InteractiveSheet(df=results_df)

https://docs.google.com/spreadsheets/d/12RhdAc8FCGip4LgGHUvAHahWw0iKqUWq6VYItS-jHOE/edit#gid=0


#Test 2

In [ ]:
raw = '''Here are the three key outputs:
\n\n**Professional Elevator Pitch:**\nAs an accomplished Security Program Manager with over 5 years of experience in optimizing program management, logistics, and compliance, I excel at streamlining complex operations and setting measurable goals. My expertise in driving continuous improvement and delivering strategic solutions makes me a valuable asset to any organization.
\n\n**Three Specific Improvement Themes:**\n\n1. **Enhance Quantifiable Metrics:** Some bullet points lack specific numbers or metrics, making it harder to evaluate the impact of accomplishments; adding more quantifiable data would strengthen these claims.\n2. **Varied Action Verbs:** Repeating action verbs like \"Improved\" and \"Reduced\" makes the text less engaging; incorporating a mix of action verbs would enhance readability and create a dynamic tone.\n3. **Bullet Point Organization:** Some bullet points seem unrelated to each other, creating disjointedness in the narrative; reorganizing them into logical groups or categories would improve overall flow.
\n\n**Resume Grade:**\nBased on the analysis, I would give this resume an A (92.9%) due to its strong accomplishments, clear formatting, and professional tone. The writer has demonstrated excellent expertise in security program management and has presented a well-structured narrative that effectively communicates their skills and achievements.\n"
'''

result = format_response(raw)
print("Elevator Pitch:\n", result['elevatorPitch'])
print("\nImprovement Themes:")
for idx, theme in enumerate(result['themes'], 1):
    print(f"{idx}. {theme}")
print("\nResume Grade Explanation:\n", result['explanation'])


Elevator Pitch:
 As an accomplished Security Program Manager with over 5 years of experience in optimizing program management, logistics, and compliance, I excel at streamlining complex operations and setting measurable goals. My expertise in driving continuous improvement and delivering strategic solutions makes me a valuable asset to any organization.

Improvement Themes:
1. Enhance Quantifiable Metrics: Some bullet points lack specific numbers or metrics, making it harder to evaluate the impact of accomplishments; adding more quantifiable data would strengthen these claims.
2. Varied Action Verbs: Repeating action verbs like "Improved" and "Reduced" makes the text less engaging; incorporating a mix of action verbs would enhance readability and create a dynamic tone.
3. Bullet Point Organization: Some bullet points seem unrelated to each other, creating disjointedness in the narrative; reorganizing them into logical groups or categories would improve overall flow.

Resume Grade Expla

In [ ]:
raw = '''Here are the three key outputs:
\n\nA professional elevator pitch based on the resume text: \nAs a creative and social individual with experience in photography and customer service, I am looking for a fun and engaging role that utilizes my great sense of humor and style. With a strong educational foundation and relevant work experience, I am excited to bring my skills and enthusiasm to a new opportunity.
\n\nThree specific improvement themes: \nThe resume could benefit from more specific examples of accomplishments and achievements in previous roles, rather than just listing job responsibilities. \nThe objective statement and some of the bullet points could be rephrased to sound more professional and less casual. \nAdding more relevant sections such as skills or volunteer work could help to round out the resume and showcase a wider range of experiences.
\n\nA brief explanation of the resume grade: \nThe resume grade of B reflects a good foundation in terms of education and work experience, but is brought down by some unprofessional language and a lack of specific accomplishments. Overall, the resume shows potential but could benefit from some revisions to make it more effective.\n",'''

result = format_response(raw)
print("Elevator Pitch:\n", result['elevatorPitch'])
print("\nImprovement Themes:")
for idx, theme in enumerate(result['themes'], 1):
    print(f"{idx}. {theme}")
print("\nResume Grade Explanation:\n", result['explanation'])

Elevator Pitch:
 As a creative and social individual with experience in photography and customer service, I am looking for a fun and engaging role that utilizes my great sense of humor and style. With a strong educational foundation and relevant work experience, I am excited to bring my skills and enthusiasm to a new opportunity.

Improvement Themes:
1. The resume could benefit from more specific examples of accomplishments and achievements in previous roles, rather than just listing job responsibilities.
2. The objective statement and some of the bullet points could be rephrased to sound more professional and less casual.
3. Adding more relevant sections such as skills or volunteer work could help to round out the resume and showcase a wider range of experiences.

Resume Grade Explanation:
 The resume grade of B reflects a good foundation in terms of education and work experience, but is brought down by some unprofessional language and a lack of specific accomplishments. Overall, the 

In [ ]:
raw = '''Here are the three key outputs:\n\nAs a highly motivated and creative individual with experience in photography and customer service, I am excited to bring my skills and enthusiasm to a dynamic team. With a strong foundation in communication and social media, I am confident in my ability to make a positive impact.
\n\nThree specific improvement themes are: \nThe resume requires a more professional objective statement that aligns with the candidate's career goals and experience. \nThe work experience section needs more specific examples and achievements that demonstrate the candidate's skills and accomplishments. \nThe education section should be more detailed, including relevant coursework or academic achievements that support the candidate's career aspirations.
\n\nThe resume grade of D+ indicates that the resume requires significant improvement in terms of formatting, content, and overall effectiveness in showcasing the candidate's skills and experience. The low grade is largely due to the lack of specific examples, weak bullet points, and unprofessional tone.\n",'''

result = format_response(raw)
print("Elevator Pitch:\n", result['elevatorPitch'])
print("\nImprovement Themes:")
for idx, theme in enumerate(result['themes'], 1):
    print(f"{idx}. {theme}")
print("\nResume Grade Explanation:\n", result['explanation'])

Elevator Pitch:
 As a highly motivated and creative individual with experience in photography and customer service, I am excited to bring my skills and enthusiasm to a dynamic team. With a strong foundation in communication and social media, I am confident in my ability to make a positive impact.

Improvement Themes:
1. The resume requires a more professional objective statement that aligns with the candidate's career goals and experience.
2. The work experience section needs more specific examples and achievements that demonstrate the candidate's skills and accomplishments.
3. The education section should be more detailed, including relevant coursework or academic achievements that support the candidate's career aspirations.

Resume Grade Explanation:
 The resume grade of D+ indicates that the resume requires significant improvement in terms of formatting, content, and overall effectiveness in showcasing the candidate's skills and experience. The low grade is largely due to the lack o

In [ ]:
raw = '''Here are the three key outputs:\n\nAs a creative and social individual with experience in photography and customer service, I am excited to bring my skills and enthusiasm to a fun and dynamic work environment. With a strong foundation in communication and a keen sense of humor, I am confident in my ability to make a positive impact in a team.
\n\nHere are three specific improvement themes: \nThe resume could benefit from more descriptive and action-oriented language to help Sally's skills and experiences stand out. \nThe objective and summary sections could be more tailored to specific job targets and qualifications to increase the effectiveness of the resume. \nThe work experience section would be more impactful with quantifiable achievements and specific examples of how Sally's skills and accomplishments added value to her previous roles.
\n\nThe resume grade of B indicates that while Sally has a good foundation in terms of education and work experience, there are areas for improvement in terms of presentation, clarity, and impact. With some refinement and attention to detail, Sally's resume could be even more effective in showcasing her strengths and helping her achieve her career goals.\n"'''

result = format_response(raw)
print("Elevator Pitch:\n", result['elevatorPitch'])
print("\nImprovement Themes:")
for idx, theme in enumerate(result['themes'], 1):
    print(f"{idx}. {theme}")
print("\nResume Grade Explanation:\n", result['explanation'])

Elevator Pitch:
 As a creative and social individual with experience in photography and customer service, I am excited to bring my skills and enthusiasm to a fun and dynamic work environment. With a strong foundation in communication and a keen sense of humor, I am confident in my ability to make a positive impact in a team.

Improvement Themes:
1. The resume could benefit from more descriptive and actionoriented language to help Sally's skills and experiences stand out.
2. The objective and summary sections could be more tailored to specific job targets and qualifications to increase the effectiveness of the resume.
3. The work experience section would be more impactful with quantifiable achievements and specific examples of how Sally's skills and accomplishments added value to her previous roles.

Resume Grade Explanation:
 The resume grade of B indicates that while Sally has a good foundation in terms of education and work experience, there are areas for improvement in terms of pres